## Asignment 7

In [1]:
# 1a. Display the first and last names of all actors from the table actor.

use sakila;
select first_name, last_name
from actor;

# 1b. Display the first and last name of each actor in a single column in upper case letters. Name the column Actor Name.
SELECT CONCAT(first_name, ' ', last_name)
AS 'Actor Name'
FROM actor;

SELECT
CONCAT(LEFT(first_name, 1)), SUBSTRING(LOWER(first_name), 2)) AS 'First Name',
CONCAT(LEFT(last_name, 1), SUBSTRING(LOWER(last_name), 2)) AS 'Last Name'
from actor;


# 2a. You need to find the ID number, first name, and last name of an actor, of whom you know only the first name, "Joe." 
# What is one query would you use to obtain this information?
SELECT actor_id, last_name, first_name
FROM actor
WHERE first_name = 'Joe';

# 2b. Find all actors whose last name contain the letters GEN:
SELECT actor_id, last_name, first_name 
FROM actor
WHERE last_name LIKE '%GEN%';

# 2c. Find all actors whose last names contain the letters LI. This time, order the rows by last name and first name, in that order:
SELECT actor_id, last_name, first_name
FROM actor
WHERE last_name LIKE '%LI%'
ORDER BY last_name ASC, first_name ASC;

# 2d. Using IN, display the country_id and country columns of the following countries: Afghanistan, Bangladesh, and China:
SELECT country_id, country
FROM country
WHERE country IN ('Afghanistan', 'Bangladesh', 'China');

# 3a. You want to keep a description of each actor. 
# You don't think you will be performing queries on a description, so create a column in the table actor named description and use the data type BLOB 
# (Make sure to research the type BLOB, as the difference between it and VARCHAR are significant).
SET SQL_SAFE_UPDATES = 0;
ALTER TABLE actor
ADD description BLOB;
SELECT * FROM actor;
SET SQL_SAFE_UPDATES = 1;

# 3b. Very quickly you realize that entering descriptions for each actor is too much effort. Delete the description column.
SET SQL_SAFE_UPDATES = 0;
ALTER TABLE actor
DROP COLUMN description;
SET SQL_SAFE_UPDATES = 1;

# 4a. List the last names of actors, as well as how many actors have that last name.
SELECT last_name, count(last_name) AS 'Total Count'
FROM actor
GROUP BY last_name;

# 4b. List last names of actors and the number of actors who have that last name, but only for names that are shared by at least two actors
SELECT last_name, COUNT(last_name) AS 'Total Count'
FROM actor
GROUP BY last_name
HAVING COUNT(last_name) > 1;

# 4c. The actor HARPO WILLIAMS was accidentally entered in the actor table as GROUCHO WILLIAMS. Write a query to fix the record.
SET SQL_SAFE_UPDATES = 0;

UPDATE actor SET first_name = 'HARPO'
WHERE actor_id = 172;

SET SQL_SAFE_UPDATES = 1;

SELECT *
FROM actor
WHERE last_name = 'Williams';

# 4d. Perhaps we were too hasty in changing GROUCHO to HARPO. 
# It turns out that GROUCHO was the correct name after all! In a single query, 
# if the first name of the actor is currently HARPO, change it to GROUCHO.
SET SQL_SAFE_UPDATES = 0;

UPDATE actor SET first_name = 'GROUCHO'
WHERE actor_id = 172;

SET SQL_SAFE_UPDATES = 1;

SELECT *
FROM actor
WHERE last_name = 'Williams';

# 5a. You cannot locate the schema of the address table. Which query would you use to re-create it?
SHOW CREATE TABLE address;
DESCRIBE address;

# 6a. Use JOIN to display the first and last names, as well as the address, of each staff member. Use the tables staff and address:
SELECT first_name, last_name, address
FROM staff
INNER JOIN address USING (address_id);

# 6b. Use JOIN to display the total amount rung up by each staff member in August of 2005. Use tables staff and payment.
SELECT last_name, first_name, SUM(amount) AS 'Total Amount'
FROM payment
INNER JOIN staff USING (staff_id)
GROUP BY payment.staff_id;

# 6c. List each film and the number of actors who are listed for that film. Use tables film_actor and film. Use inner join.
SELECT film.title, COUNT(actor_id) AS 'Number of Actor'
FROM film
INNER JOIN film_actor USING (film_id)
GROUP BY film.title;

#  6d. How many copies of the film Hunchback Impossible exist in the inventory system?
SELECT film.title, COUNT(inventory_id) AS 'Copies in Inventory'
FROM inventory
INNER JOIN film USING (film_id)
WHERE film.title = 'Hunchback Impossible';

# 6e. Using the tables payment and customer and the JOIN command, list the total paid by each customer. 
# List the customers alphabetically by last name:
SELECT last_name, first_name, SUM(amount) AS 'Total Amount Paid'
FROM payment
INNER JOIN customer USING (customer_id)
GROUP BY payment.customer_id
ORDER BY customer.last_name ASC;

# 7a. The music of Queen and Kris Kristofferson have seen an unlikely resurgence. 
# As an unintended consequence, films starting with the letters K and Q have also soared in popularity.
# Use subqueries to display the titles of movies starting with the letters K and Q whose language is English.
SELECT title
FROM film
WHERE title LIKE 'K%' or title LIKE 'Q%' AND language_id = (
	SELECT language_id
    FROM language
    WHERE name = 'English');

# 7b. Use subqueries to display all actors who appear in the film Alone Trip.
SELECT last_name, first_name
FROM actor
WHERE actor_id IN (
	SELECT actor_id
    FROM film_actor
    WHERE film_id = (
		SELECT film_id
        FROM film
        WHERE title = 'Alone Trip')
);

# 7c. You want to run an email marketing campaign in Canada, for which you will need the names and email addresses of all Canadian customers. 
# Use joins to retrieve this information.
SELECT last_name, first_name, email
FROM customer
WHERE address_id IN (
	SELECT address_id
    FROM address
    WHERE city_id IN (
		SELECT city_id
        FROM city
        WHERE country_id = (
			SELECT country_id
            FROM country
            WHERE country = 'Canada')
));

# 7d. Sales have been lagging among young families, and you wish to target all family movies for a promotion. 
# Identify all movies categorized as family films.
SELECT title
FROM film
WHERE film_id IN (
	SELECT film_id
    FROM film_category
    WHERE category_id = (
		SELECT category_id
        FROM category
        WHERE name = 'Family')
);

# 7e. Display the most frequently rented movies in descending order.
SELECT film.title, COUNT(rental_id) AS 'Rent Frequency'
FROM inventory
INNER JOIN film USING (film_id)
INNER JOIN rental USING (inventory_id)
GROUP BY film.film_id
ORDER BY COUNT(rental_id);

# 7f. Write a query to display how much business, in dollars, each store brought in.
SELECT store_id AS 'Store', SUM(amount) AS 'Total Revenue'
FROM staff
INNER JOIN payment USING (staff_id)
INNER JOIN store USING (store_id)
GROUP BY store.store_id;

# 7g. Write a query to display for each store its store ID, city, and country.
SELECT store_id AS 'Store', 
(SELECT city
FROM city
WHERE city_id IN (
	SELECT city_id
    FROM address
    WHERE address_id IN (
		SELECT address_id 
        FROM store))),
(SELECT country
FROM country
WHERE country_id IN (
	SELECT country_id
	FROM city
	WHERE city_id IN (
		SELECT city_id
		FROM address
		WHERE address_id IN (
			SELECT address_id 
			FROM store))))
FROM store;


SELECT city
FROM city
WHERE city_id IN (
	SELECT city_id
    FROM address
    WHERE address_id IN (
		SELECT address_id 
        FROM store));

SELECT country
FROM country
WHERE country_id IN (
	SELECT country_id
	FROM city
	WHERE city_id IN (
		SELECT city_id
		FROM address
		WHERE address_id IN (
			SELECT address_id 
			FROM store)));

SELECT store_id AS 'Store'
FROM store;

SELECT store_id, city, country
FROM store
INNER JOIN address USING (address_id)
INNER JOIN city ON address.city_id = city.city_id
INNER JOIN country ON city.country_id = country.country_id
GROUP BY store.store_id;


# 7h. List the top five genres in gross revenue in descending order. 
# (Hint: you may need to use the following tables: category, film_category, inventory, payment, and rental.)
SELECT name AS 'Genres', 
(SELECT SUM(amount)
FROM payment
WHERE rental_id IN (
	SELECT rental_id
    FROM Rental
    WHERE inventory_id IN (
		SELECT inventory_id 
        FROM inventory
        WHERE film_id IN (
			SELECT film_id
            FROM film_category
            GROUP BY category_id)))) AS 'Total Revenue'
FROM category;

SELECT name AS 'Genres', SUM(amount) AS 'Total Revenue'
FROM payment
INNER JOIN rental USING (rental_id)
INNER JOIN inventory ON rental.inventory_id = inventory.inventory_id
INNER JOIN film_category ON inventory.film_id = film_category.film_id
INNER JOIN category ON film_category.category_id = category.category_id
GROUP BY category.name
ORDER BY SUM(amount) DESC LIMIT 5;

# 8a. In your new role as an executive, you would like to have an easy way of viewing the Top five genres by gross revenue. 
# Use the solution from the problem above to create a view. If you haven't solved 7h, you can substitute another query to create a view.
CREATE VIEW gross_revenue AS
SELECT name AS 'Genres', SUM(amount) AS 'Total Revenue'
FROM payment
INNER JOIN rental USING (rental_id)
INNER JOIN inventory ON rental.inventory_id = inventory.inventory_id
INNER JOIN film_category ON inventory.film_id = film_category.film_id
INNER JOIN category ON film_category.category_id = category.category_id
GROUP BY category.name
ORDER BY SUM(amount) DESC LIMIT 5;

# 8b. How would you display the view that you created in 8a?
SELECT * from gross_revenue;

# 8c. You find that you no longer need the view top_five_genres. Write a query to delete it.
DROP VIEW gross_revenue;







































SyntaxError: invalid syntax (<ipython-input-1-6131a303fedb>, line 3)